In [2]:
import itertools

import numpy as np
from scipy.spatial.transform import Rotation

import meshplot as mp
import triangle as tr

In [3]:
shading = {
    "flat": True, # Flat or smooth shading of triangles
    "wireframe": True, "wire_width": 0.1, "wire_color": "black", # Wireframe rendering
    "width": 600, "height": 600, # Size of the viewer canvas
    "antialias": True, # Antialising, might not work on all GPUs
    "scale": 2.0, # Scaling of the model
    "side": "DoubleSide", # FrontSide, BackSide or DoubleSide rendering of the triangles
    "colormap": "viridis", "normalize": [None, None], # Colormap and normalization for colors
    "background": "#ffffff", # Background color of the canvas
    "line_width": 2.0, "line_color": "black", # Line properties of overlay lines
    "bbox": False, # Enable plotting of bounding box
    "point_color": "red", "point_size": 0.05 # Point properties of overlay points
}

In [4]:
def regular_2D_grid(n):
    delta = 1 / (n - 1)
    # map from (i, j) coordinates to vertex id
    ij2v = np.full((n, n), -1)
    V = []

    # Corner Vertices
    V.extend([[0, 0], [1, 0], [0, 1]])
    ij2v[0, 0] = 0
    ij2v[-1, 0] = 1
    ij2v[0, -1] = 2

    # Edge vertices
    # [0, 0] -> [1, 0]
    ij2v[1:-1, 0] = np.arange(n - 2) + 3
    V.extend([[x, 0] for x in np.linspace(0, 1, n)[1:-1]])
    # [1, 0] -> [0, 1]
    ij2v[np.arange(n - 2, 0, -1), np.arange(1, n - 1)] = (
        np.arange(n - 2) + (3 + n - 2))
    V.extend([[1 - x, x] for x in np.linspace(0, 1, n)[1:-1]])
    # [0, 1] -> [0, 0]
    ij2v[0, -2:0:-1] = np.arange(n - 2) + (3 + 2 * (n - 2))
    V.extend([[0, 1 - y] for y in np.linspace(0, 1, n)[1:-1]])

    # Interior vertices
    for j in range(1, n - 1):
        for i in range(1, n - 1):
            if i + j >= n - 1:
                break
            ij2v[i, j] = len(V)
            V.append([i * delta, j * delta])

    # Create triangulated faces
    F = []
    for i, j in itertools.product(range(n - 1), range(n - 1)):
        for f in [ij2v[[i, i + 1, i], [j, j, j + 1]],
                  ij2v[[i + 1, i + 1, i], [j, j + 1, j + 1]]]:
            if (f >= 0).all():
                F.append(f)

    return np.array(V), np.array(F)

In [5]:
in_V = np.array([
    [0.4987804 , 0.77914574, 0],
    [0.41414302, 0.66677665, 1],
    [0.31142051, 0.14120175, 0]
])
# in_V = np.array([
#     [0, 0],
#     [1, 0],
#     [0, 1]
# ])

in_E = np.array([[0, 1], [1, 2], [2, 0]])

In [6]:
n_ref = 3
V_grid, F_grid = regular_2D_grid(n_ref+2)
V_reg = []
for alpha, beta in V_grid:
    V_reg.append((1 - alpha - beta) * in_V[0] + alpha * in_V[1] + beta * in_V[2])
V_reg = np.vstack(V_reg)

In [7]:
plt = mp.plot(V_reg, F_grid, shading=shading)
plt.add_points(V_reg, shading=shading)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4051004…

1

In [8]:
def refine_edges(in_V, in_E, max_edge_len):
    # n_ref = 3
    V = in_V.tolist()
    E = []
    for i in range(in_V.shape[0]):
        v1, v0 = in_V[in_E[i]]
        n_ref = int(np.ceil(np.linalg.norm(v1 - v0) / max_edge_len))
        alphas = np.linspace(0, 1, n_ref+2)[1:-1]
        E.append([in_E[i, 0], len(V)])
        for ai, a in enumerate(alphas):
            if ai < n_ref - 1:
                E.append([len(V), len(V) + 1])
            V.append((a * (v1 - v0) + v0).tolist())
        E.append([len(V) - 1, in_E[i, 1]])

    return np.vstack(V), np.vstack(E)

In [9]:
max_edge_len = 0.1
p = 3 * max_edge_len / 2
max_area = np.sqrt(p * (p - max_edge_len)**3)

V, E = refine_edges(in_V, in_E, max_edge_len)

R = Rotation.align_vectors(
    np.array([[0, 0, 1]]),
    np.cross(in_V[1] - in_V[0], in_V[2] - in_V[0]).reshape(1, 3))[0].as_matrix()

V_2D = V @ R.T
z = V_2D[:, 2].copy()
assert((abs(z - z[0]) < 1e-12).all())
z = z[0]
V_2D = V_2D[:, :2]
t = tr.triangulate({"vertices": V_2D, "segments": E}, f'Ya{max_area:f}q')

V_out = np.hstack([t["vertices"], np.full((t["vertices"].shape[0], 1), z)]) @ R
F_out = t["triangles"]

plt = mp.plot(V_out, F_out, shading=shading)
plt.add_points(V_out, shading=shading)
plt.add_edges(V, E, shading=(shading | {"line_color": "blue", "line_width": 2}))
plt.add_points(V, shading=(shading | {"point_color": "blue"}))

/var/folders/y1/l7pg454j0jv6b028zqg1frsc0000gn/T/ipykernel_87164/3900946000.py:7: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  R = Rotation.align_vectors(


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4051004…

3